# Number of Feeds with Fares v2 data

[GH issue](https://github.com/cal-itp/data-infra/issues/552)

* The total number of feeds that have a non-empty `fare_leg_rules.txt` file.
* The MVP for checking this however is to simply assert that the `fare_leg_rules.txt` file is present and it contains at least one potentially empty row.

In [5]:
import altair as alt
import altair_saver
import pandas as pd
import os
#os.environ["CALITP_BQ_MAX_BYTES"] = str(100_000_000_000)

import calitp
from calitp.tables import tbl
from siuba import *

from shared_utils import altair_utils, geography_utils, styleguide
from shared_utils import calitp_color_palette as cp

alt.renderers.enable('altair_saver', fmts=['png'])

DATA_PATH = "./data/"
IMG_PATH = "./img/"

In [6]:
dim_feeds = (tbl.views.gtfs_schedule_dim_feeds()
    >> select(_.feed_key, 
              _.calitp_itp_id, _.calitp_url_number, 
              _.calitp_agency_name, 
              _.calitp_feed_name, _.feed_publisher_name,
             )
)

fares_feeds = (
    tbl.views.gtfs_schedule_fact_daily_feed_files()
    >> filter(_.file_key=="fare_leg_rules.txt", 
              _.is_interpolated == False)
    >> select(_.feed_key, _.date, )
    >> inner_join(_, dim_feeds, "feed_key")
    >> collect()
)

In [7]:
feeds_by_date = (geography_utils.aggregate_by_geography(
        fares_feeds,
        group_cols = ["date"],
        count_cols = ["feed_key"]
    )
)

feeds_by_date = (feeds_by_date
                 .assign(
                     date = pd.to_datetime(feeds_by_date.date),
                 ).sort_values("date")
                 .reset_index(drop=True)
                )

feeds_by_date.head()

,date,feed_key
0,2021-07-31,1
1,2021-08-01,1
2,2021-08-03,1
3,2021-08-04,1
4,2021-08-05,1


In [9]:
axis_date_format ="%-m/%-d/%y"


chart = (alt.Chart(feeds_by_date)
         .mark_line()
         .encode(
            x=alt.X("date", axis=alt.Axis(format=axis_date_format)),
            y=alt.Y("feed_key", title="# feeds"), 
            color=alt.value(cp.CALITP_CATEGORY_BRIGHT_COLORS[0])
    )
)

chart = (styleguide.preset_chart_config(chart)
         .properties(title="Number of Feeds with fare_leg_rules.txt")
        )

chart.save(f"{IMG_PATH}farev2_feeds.png")

![farev2_feeds](./img/farev2_feeds.png)

In [10]:
feeds_by_date.feed_key.sum()

8556